In [4]:
! git clone https://github.com/kaspergroenbek98/first-first-year-project.git

fatal: destination path 'first-first-year-project' already exists and is not an empty directory.


In [5]:
import numpy as np
import nltk
import csv
import nltk
#from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))


# Read the data into a header and a data np.array
with open('first-first-year-project/data.csv', encoding='latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = np.array([np.array(line) for line in csv_reader])
header, data = data[0,:], data[1:,:]
print(header)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['other_topic' 'resolution_topics' 'gender' 'name' 'Resolution_Category'
 'retweet_count' 'text' 'tweet_coord' 'tweet_created' 'tweet_date'
 'tweet_id' 'tweet_location' 'tweet_state' 'user_timezone' 'tweet_region']


In [0]:
porter = nltk.PorterStemmer()

# Remove all stopwords, non-alphabet words (except spaces), and stem the words
for i, row in enumerate(data[:,6]):
    row = row.lower()
    row = ''.join(char for char in row if char.isalpha() or char == ' ')
    row = ' '.join(porter.stem(word) for word in row.split() if word not in stopwords)
    data[i,6] = row

# Create a long string of ALL words, then token them, and create a frequency distribution
actualWords = ' '.join(token for sentence in data[:,6] for token in sentence.split())
tokens = nltk.word_tokenize(actualWords)
fd = nltk.FreqDist(tokens)


In [0]:
word_features = [word for (word, count) in fd.most_common(2000)]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features
size = 1000
featuresets = [(document_features(d.split()), c) for (d,c) in zip(data[:size,6], data[:size,-1])] # column 6 is text data, column 2 is gender data
train_set, test_set = featuresets[size//2:], featuresets[:size//2]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [8]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.284
Most Informative Features
          contains(want) = True           Midwes : South  =      5.8 : 1.0
         contains(happi) = True           Northe : South  =      5.6 : 1.0
          contains(diet) = True           Northe : West   =      5.1 : 1.0
  contains(happynewyear) = True           Midwes : South  =      5.1 : 1.0
         contains(thing) = True           Midwes : West   =      4.7 : 1.0
         contains(bitch) = True           Northe : South  =      4.5 : 1.0
          contains(best) = True           Northe : Midwes =      4.5 : 1.0
 contains(mlbpaclubhous) = True           Midwes : South  =      4.4 : 1.0
           contains(nye) = True           Northe : West   =      4.2 : 1.0
           contains(let) = True           Midwes : Northe =      4.1 : 1.0


In [0]:
bigrm = list(nltk.ngrams(tokens, 2))
fdbigrm = nltk.FreqDist(bigrm)


In [0]:
word_features = list(fdbigrm.keys())[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features[f'contains({word})'] = (word in document_words)
    return features
size = 1000
featuresets = [(document_features(nltk.ngrams(d.split(), 2)), c) for (d,c) in zip(data[:size,6], data[:size,-1])] # column 6 is text data, column 2 is gender data
train_set, test_set = featuresets[size//2:], featuresets[:size//2]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [11]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.264
Most Informative Features
contains(('newyearsresolut', 'happynewyear')) = True           Midwes : South  =      3.9 : 1.0
contains(('newyearsresolut', 'get')) = True            South : Northe =      3.7 : 1.0
contains(('newyearsresolut', 'stay')) = True           Midwes : West   =      3.6 : 1.0
contains(('newyearsresolut', 'becom')) = True            South : West   =      3.4 : 1.0
contains(('newyearsresolut', 'make')) = True           Northe : West   =      3.2 : 1.0
contains(('newyearsresolut', 'go')) = True             West : Northe =      3.1 : 1.0
contains(('year', 'resolut')) = True             West : Northe =      3.0 : 1.0
contains(('newyearsev', 'newyearsresolut')) = True           Midwes : South  =      2.8 : 1.0
contains(('new', 'year')) = True             West : Northe =      2.5 : 1.0
contains(('newyearsresolut', 'stop')) = True           Midwes : West   =      2.4 : 1.0


In [12]:
'newyearsresolution' in stopwords

False